In [ ]:
import sys
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.integrate
import math
import pandas as pd
import itertools
import random
import statsmodels.api as sm
import time

In [ ]:
#Compute expectation and variance of Z random variable parameterized by m and v

def z_expectation_variance(m,v):
    #m is question mean
    #v is question variance
    integ_bound = 30.0
    
    #Set up functions for calculating expectation and variance of Z
    fun1 = lambda z: ((1 + math.e**(-m - math.sqrt(v)*z))**(-1))*(math.e**((-z**2)/2))/math.sqrt(2*math.pi)
    fun2 = lambda z: z*((1 + math.e**(-m - math.sqrt(v)*z))**(-1))*(math.e**((-z**2)/2))/math.sqrt(2*math.pi)
    fun3 = lambda z: z*z*((1 + math.e**(-m - math.sqrt(v)*z))**(-1))*(math.e**((-z**2)/2))/math.sqrt(2*math.pi)
    
    #Calculate expectation and variance of Z. C is the normalization constant that ensures the pdf of Z
    #integrates to be 1. 
    C = scipy.integrate.quad(fun1, -integ_bound, integ_bound)[0]
    mu_z = scipy.integrate.quad(fun2, -integ_bound, integ_bound)[0]/C
    var_z = (scipy.integrate.quad(fun3, -integ_bound, integ_bound)[0] / C) - mu_z**2
    
    return [mu_z, var_z]

In [ ]:
#This function is used to generate an nxn matrix M(r) such that M_ij = r^|i-j| for -1<r<1. This matrix is called a 
#Kac-Murdock-Szego matrix.
def KMS_Matrix(n,r):
    #n: this is the number of rows and columns of the matrix
    #r: this is the coefficient given above that determines the value of the matrice's entries 
    
    
    M = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            M[i,j] = r**abs(i-j)
        
    return M

In [ ]:
#Define a set that has all the differences between binary products

def product_diff_list(n):
    #n: the number of attributes of the products
    
    #Example of itertools.product:
    #itertools.product(range(2), repeat = 3) --> 000 001 010 011 100 101 110 111
    p_d_l = list(itertools.product([-1.0,0.0,1.0],repeat = n))
    
    #Return the index of the tuple with all 0s.
    zero_index = p_d_l.index(tuple([0]*n))

    #Note that at this point, product_diff_list contains some redundant information. Due to
    #the symmetry of the one-step and two-step acquisition function in terms of question mean, question pairs such as 
    #(-1,-1,-1,...,-1) and (1,1,1,...,1) (i.e. negative multiples) will evaluate as the same under the one-step and two-step
    #acquisition functions. Due to the structure of product_diff_list, we can remove every question pair before and including
    #the question pair with all zero entries in order to remove this redundant information.
    for i in range(0,zero_index + 1):
        p_d_l.pop(0)
    
    p_d_l = [np.array(a) for a in p_d_l]
    return p_d_l

In [ ]:
#Given a trinary vector of 0, 1, and -1, find two binary products whose difference is the trinary vector.
def question_extractor(prod):
    #prod: This is a trinary vector of 0, 1, and -1 that represents the difference between two products, which
    #are represented by binary vectors.
    
    x = [0]*len(prod)
    y = [0]*len(prod)
    for i in range(0,len(prod)):
        if prod[i] == 1.0:
            x[i] = 1.0
            y[i] = 0.0
        if prod[i] == 0.0:
            x[i] = 0.0
            y[i] = 0.0
        if prod[i] == -1.0:
            x[i] = 0.0
            y[i] = 1.0
    return x,y

In [ ]:
def moment_matching_update(x,y,mu_prior,Sig_prior):
    #x and y are a question pair, x is preferred over y.
    #mu_prior and Sig_prior are expectation and covariance matrix
    #Make sure x, y, mu_prior, and Sig_prior are numpy arrays
    x_vec = np.array(x)
    y_vec = np.array(y)
    mu_prior_vec = np.array(mu_prior)
    Sig_prior_vec = np.array(Sig_prior)
    
    #Define question expectation and question variance
    v = x_vec - y_vec
    mu_v = np.dot(mu_prior_vec,v)
    Sig_dot_v = np.dot(Sig_prior_vec,v)
    Sig_v = np.dot(v,Sig_dot_v)
    
    #Save np.dot(Sig_prior_vec,v) as a variable (DONE)
    
    #Calculate expectation and variance for Z random variable
    
    [mu_z, var_z] = z_expectation_variance(mu_v,Sig_v)
    
    
    #Calculate the update expectation and covariance matrix for 
    #posterior
    mu_posterior = mu_prior_vec + (mu_z/math.sqrt(Sig_v))*Sig_dot_v
    Sig_posterior = ((var_z-1)/Sig_v)*np.outer(Sig_dot_v,Sig_dot_v) + Sig_prior_vec
    
    return mu_posterior, Sig_posterior

In [ ]:
#This function constructs a batch design based off of average question mean, average question variance, and average
#question orthogonality. For the average question orthogonality, we take the absolute value of the summands rather than
#the square. We also normalize mu and Sig in the objective so that we do not need to keep on refitting the parameters 
#that go with question mean, question variance, and question orthogonality.

def batch_design_AO(mu,Sig,batch_size,quest_mean_log_coeff,quest_var_log_coeff,quest_orth_log_coeff,t_lim = 100):
    #mu: expectation of prior on beta
    #Sig: Covariance matrix of prior on beta
    #batch_size: the number of questions we want to return in our batch design. This should be less or equal to the number
    #of attributes
    #quest_mean_log_coeff: this is a fitting parameter that goes with the average question mean and is obtained 
    #by fitting a linear model log (D-err/Init_det) ~ AM/||l*mu|| + AV/||s*Sig|| + AO/||s*Sig|| + ||l*mu|| + ||s*Sig|| and using the fitted parameter that goes with
    #AM/||l*mu||
    #quest_var_log_coeff: this is a fitting parameter that goes with the average question variance and is obtained 
    #by fitting a linear model log (D-err/Init_det) ~ AM/||l*mu|| + AV/||s*Sig|| + AO/||s*Sig|| + ||l*mu|| + ||s*Sig|| and using the fitted parameter that goes with
    #AV/||s*Sig||
    #quest_orth_log_coeff: this is a fitting parameter that goes with the average question orthogonality and is obtained 
    #by fitting a linear model log (D-err/Init_det) ~ AM/||l*mu|| + AV/||s*Sig|| + AO/||s*Sig|| + ||l*mu|| + ||s*Sig|| and using the fitted parameter that goes with
    #AO/||s*Sig||
    #(l,s) are scaling parameters for mu and Sig that divide the space into different signal-to-noise ratio regions.
    #t_lim: this is the max amount of time we want to take to construct the batch
    
    #This is the number of attributes for the products
    n = len(Sig[0])
    
    m = gp.Model("mip1")
    m.setParam('Timelimit',t_lim)
    #Only save logfile for this experiment
    m.setParam('LogFile',"Batch_AO_batchsize"+str(batch_size)+"_meancoeff_"+str(quest_mean_log_coeff)+"_varcoeff_"+
               str(quest_var_log_coeff)+"_orthcoeff_"+str(quest_orth_log_coeff))
    
    #calculate 2-norms of mu and Sigma
    mu_2norm = np.linalg.norm(mu,2)
    Sig_2norm = np.linalg.norm(Sig,2)
    
    #List of tuples for delta variable
    delta_tuples = []
    for i in range(batch_size):
        for j in range(i+1,batch_size):
            delta_tuples.append((i,j))
    
    #Set up the x_i and y_i, i = 1,...,batchsize
    X = m.addMVar((batch_size,n),vtype = GRB.BINARY)
    Y = m.addMVar((batch_size,n),vtype = GRB.BINARY)
    Delta = m.addVars(delta_tuples, lb=0.0, vtype = GRB.CONTINUOUS)
    
    #Set up the objective function.
    m.setObjective((quest_mean_log_coeff/(batch_size*mu_2norm))*sum([mu@X[i] - mu@Y[i] for i in range(batch_size)]) + 
                   (quest_var_log_coeff/(batch_size*Sig_2norm))*sum([X[i]@Sig@X[i] - X[i]@(2.0*Sig)@Y[i] + 
                   Y[i]@Sig@Y[i] for i in range(batch_size)]) + 
                   sum([(quest_orth_log_coeff/(batch_size*(batch_size-1)*Sig_2norm/2))*Delta[i,j] for i in range(batch_size) for j in range(i+1,batch_size)]),GRB.MINIMIZE)
    
    #Set up the constraints that force the products in question i to be different, as well as forcing the symmetry
    #exploitation condition.
    for i in range(batch_size):
        m.addConstr(X[i]@X[i] - X[i]@Y[i] - Y[i]@X[i] + Y[i]@Y[i] >= 1)
        m.addConstr(mu@X[i] - mu@Y[i] >= 0)
        
    #Set up the Sigma-orthogonality constraint for all questions i and j, i not equal to j. Also add constraints
    #to make sure that questions within a batch are different, including with respect to switching order of products in
    #the questions.
    for i in range(batch_size):
        for j in range(i+1,batch_size):
            m.addConstr(X[i]@Sig@X[j] - X[i]@Sig@Y[j] - Y[i]@Sig@X[j] + Y[i]@Sig@Y[j] - Delta[i,j] <= 0)
            m.addConstr(X[i]@Sig@X[j] - X[i]@Sig@Y[j] - Y[i]@Sig@X[j] + Y[i]@Sig@Y[j] + Delta[i,j] >= 0)
            m.addConstr(X[i]@X[i] - X[i]@Y[i] - X[i]@X[j] + X[i]@Y[j] -
                       Y[i]@X[i] + Y[i]@Y[i] + Y[i]@X[j] - Y[i]@Y[j] -
                       X[j]@X[i] + X[j]@Y[i] + X[j]@X[j] - X[j]@Y[j] +
                       Y[j]@X[i] - Y[j]@Y[i] - Y[j]@X[j] + Y[j]@Y[j] >= 1)
            m.addConstr(X[i]@X[i] - X[i]@Y[i] - X[i]@Y[j] + X[i]@X[j] -
                       Y[i]@X[i] + Y[i]@Y[i] + Y[i]@Y[j] - Y[i]@X[j] -
                       Y[j]@X[i] + Y[j]@Y[i] + Y[j]@Y[j] - Y[j]@X[j] +
                       X[j]@X[i] - X[j]@Y[i] - X[j]@Y[j] + X[j]@X[j] >= 1)
            
    m.optimize()
    
    #This will be the list of products
    Q = [ [] for i in range(batch_size)]
    D = [ [] for i in range(batch_size-1)]
    
    for i in range(batch_size):
        Q[i].append(X[i].X)
        Q[i].append(Y[i].X)
        
    for i in range(batch_size):
        for j in range(i+1, batch_size):
            D[i].append(Delta[i,j].X)
        
    return[Q,D]

In [ ]:
#This function constructs a batch design based off of average question mean, average question variance, and MAXIMUM
#question orthogonality. For the average question orthogonality, we take the absolute value of the summands rather than
#the square. We also normalize mu and Sig in the objective so that we do not need to keep on refitting the parameters 
#that go with question mean, question variance, and question orthogonality.

def batch_design_MO(mu,Sig,batch_size,quest_mean_log_coeff,quest_var_log_coeff,quest_orth_log_coeff,t_lim = 100):
    #mu: expectation of prior on beta
    #Sig: Covariance matrix of prior on beta
    #batch_size: the number of questions we want to return in our batch design. This should be less or equal to the number
    #of attributes
    #quest_mean_log_coeff: this is a fitting parameter that goes with the average question mean and is obtained 
    #by fitting a linear model log (D-err/Init_det) ~ AM/||l*mu|| + AV/||s*Sig|| + MO/||s*Sig|| + ||l*mu|| + ||s*Sig|| and using the fitted parameter that goes with
    #AM/||l*mu||
    #quest_var_log_coeff: this is a fitting parameter that goes with the average question variance and is obtained 
    #by fitting a linear model log (D-err/Init_det) ~ AM/||l*mu|| + AV/||s*Sig|| + MO/||s*Sig|| + ||l*mu|| + ||s*Sig|| and using the fitted parameter that goes with
    #AV/||s*Sig||
    #quest_orth_log_coeff: this is a fitting parameter that goes with the average question orthogonality and is obtained 
    #by fitting a linear model log (D-err/Init_det) ~ AM/||l*mu|| + AV/||s*Sig|| + MO/||s*Sig|| + ||l*mu|| + ||s*Sig|| and using the fitted parameter that goes with
    #MO/||s*Sig||
    #(l,s) are scaling parameters for mu and Sig that divide the space into different signal-to-noise ratio regions.
    #t_lim: this is the max amount of time we want to take to construct the batch
    
    #This is the number of attributes for the products
    n = len(Sig[0])
    
    m = gp.Model("mip1")
    m.setParam('Timelimit',t_lim)
    #Only save logfile for this experiment
    m.setParam('LogFile',"Batch_MO_batchsize"+str(batch_size)+"_meancoeff_"+str(quest_mean_log_coeff)+"_varcoeff_"+
               str(quest_var_log_coeff)+"_orthcoeff_"+str(quest_orth_log_coeff))
    
    #calculate 2-norms of mu and Sigma
    mu_2norm = np.linalg.norm(mu,2)
    Sig_2norm = np.linalg.norm(Sig,2)
    
    #Set up the x_i and y_i, i = 1,...,batchsize
    X = m.addMVar((batch_size,n),vtype = GRB.BINARY)
    Y = m.addMVar((batch_size,n),vtype = GRB.BINARY)
    delta = m.addVar(lb=0.0, vtype = GRB.CONTINUOUS)
    
    #set up the objective function
    m.setObjective((quest_mean_log_coeff/(batch_size*mu_2norm))*sum([mu@X[i] - mu@Y[i] for i in range(batch_size)]) +
                  (quest_var_log_coeff/(batch_size*Sig_2norm))*sum([X[i]@Sig@X[i] - X[i]@(2.0*Sig)@Y[i] + 
                   Y[i]@Sig@Y[i] for i in range(batch_size)]) + (quest_orth_log_coeff/Sig_2norm)*delta,GRB.MINIMIZE)
    
    #Set up the constraints that force the products in question i to be different, as well as forcing the symmetry
    #exploitation condition.
    for i in range(batch_size):
        m.addConstr(X[i]@X[i] - X[i]@Y[i] - Y[i]@X[i] + Y[i]@Y[i] >= 1)
        m.addConstr(mu@X[i] - mu@Y[i] >= 0)
    
    #Set up the Sigma-orthogonality constraint for all questions i and j, i not equal to j. Also add constraints
    #to make sure that questions within a batch are different, including with respect to switching order of products in
    #the questions.
    for i in range(batch_size):
        for j in range(i+1,batch_size):
            m.addConstr(X[i]@Sig@X[j] - X[i]@Sig@Y[j] - Y[i]@Sig@X[j] + Y[i]@Sig@Y[j] - delta <= 0)
            m.addConstr(X[i]@Sig@X[j] - X[i]@Sig@Y[j] - Y[i]@Sig@X[j] + Y[i]@Sig@Y[j] + delta >= 0)
            m.addConstr(X[i]@X[i] - X[i]@Y[i] - X[i]@X[j] + X[i]@Y[j] -
                       Y[i]@X[i] + Y[i]@Y[i] + Y[i]@X[j] - Y[i]@Y[j] -
                       X[j]@X[i] + X[j]@Y[i] + X[j]@X[j] - X[j]@Y[j] +
                       Y[j]@X[i] - Y[j]@Y[i] - Y[j]@X[j] + Y[j]@Y[j] >= 1)
            m.addConstr(X[i]@X[i] - X[i]@Y[i] - X[i]@Y[j] + X[i]@X[j] -
                       Y[i]@X[i] + Y[i]@Y[i] + Y[i]@Y[j] - Y[i]@X[j] -
                       Y[j]@X[i] + Y[j]@Y[i] + Y[j]@Y[j] - Y[j]@X[j] +
                       X[j]@X[i] - X[j]@Y[i] - X[j]@Y[j] + X[j]@X[j] >= 1)
    
    m.optimize()
    
    #This will be the list of products
    Q = [ [] for i in range(batch_size)]
    
    for i in range(batch_size):
        Q[i].append(X[i].X)
        Q[i].append(Y[i].X)
        
    return[Q,delta.X]

In [ ]:
#This function is used to generate data to estimate the parameters in the normalized AO model. The normalized AO model
#is given by log(D-err/Det^(1/2)(Sig)) ~ AM/||L*mu|| + AV/||S*Sig|| + AO/||S*Sig|| + ||L*mu|| + ||S*Sig||. AM, AV, and AO denote
#the average question mean, average quesiton variance, and average question orthogonality of a given design under prior
#N(mu, Sig). L and S denote varying signal and noise levels, respectively. The normalized AO model is used in our
#optimization procedure so that we will not have to refit the parameters in the optimization model everytime the user
#answers a batch. The idea is that varying L and S enough should encompass a wide enough range so that mu and Sig will
#be within this range after updating.
#In this function, we also include MO so that we may fit a maximum orthogonality model.

#!!! rng will need to be set before calling this function !!!

def norm_AO_MO_data_generation(init_mu, init_Sig, batch_size, L, S, num_random_batches, num_true_partworths):
    #init_mu: This is the initial expectation of the partworths.
    #init_Sig: This is the initial covariance matrix of the partworths.
    #batch_size: This is the number of questions in each batch
    #L: This is a vector which holds varying levels of signal (multiply with mu). For example,
    #we could have L = [0.25,1.0,4.0]
    #S: This is a vector which holds varying levels of noise (multiply with Sig). For example,
    #we could have S = [0.25,1.0,4.0]
    #num_random_batches: This is the number of random batches that we will generate for collecting data on log(D-err),
    #AM, AV, and AO (and MO). This set of random batches will be used for each level combination of L and S.
    #num_true_partworths: This is the number of true/baseline partworths we will use to evaluate the d-error of a design.
    
    attr_num = len(init_mu)
    
    #Create lists to store average orthogonality and max orthogonality, as well as d-error and average question mean and
    #average question variance, and ||L*mu|| and ||S*Sig|| as well.
    average_orthogonality = []
    
    maximum_orthogonality = []
    
    average_question_mean = []
    
    average_question_variance = []
    
    average_d_error = []
    
    L_mu = []
    
    S_Sig = []
    
    init_sqrt_determinant = []
    
    #Create a list of all products.
    prod_list = product_diff_list(attr_num)
    
    #Construct the set of batch designs
    batch_set = [[] for i in range(num_random_batches)]
    for i in range(num_random_batches):
        random_question_matrix = random.sample(prod_list,batch_size)
        for m in range(batch_size):
            #random_question_vec = random.sample(prod_list,1)[0]
            #[x,y] = question_extractor(random_question_vec)
            [x,y] = question_extractor(random_question_matrix[m])
            batch_set[i].append([x,y])
    
    #Record the scaled norm of mu and Sig for each combination of L and S
    for l in L:
        for s in S:
            for i in range(num_random_batches):
                L_mu.append(l*np.linalg.norm(init_mu,2))
                S_Sig.append(s*np.linalg.norm(init_Sig,2))
                init_sqrt_determinant.append(np.sqrt(np.linalg.det(s*init_Sig)))
    
    #Calculate AM, AV, AO and MO for each of the batches
    for l in L:
        for s in S:
            for i in range(num_random_batches):
                random_batch_question_mean = []
                random_batch_question_variance = []
                random_batch_orthogonality = []
        
                for p in range(batch_size):
                    x_p = np.array(batch_set[i][p][0])
                    y_p = np.array(batch_set[i][p][1])
                    random_batch_question_mean.append(np.abs(np.dot(l*init_mu,x_p - y_p)))
                    random_batch_question_variance.append(np.dot(x_p - y_p, np.dot(s*init_Sig,x_p - y_p)))
                    for q in range(p+1, batch_size):
                        x_q = np.array(batch_set[i][q][0])
                        y_q = np.array(batch_set[i][q][1])
                        random_batch_orthogonality.append(np.abs(np.dot(x_p - y_p, np.dot(s*init_Sig,x_q - y_q))))
                
                #We use this if statement in case the batch size is 1.
                if len(random_batch_orthogonality) > 0:
                    average_orthogonality.append(np.mean(np.array(random_batch_orthogonality)))
                    maximum_orthogonality.append(np.max(np.array(random_batch_orthogonality)))
        
                average_question_mean.append(np.mean(np.array(random_batch_question_mean)))
                average_question_variance.append(np.mean(np.array(random_batch_question_variance)))
            
    #Calculate the D-error.
    for l in L:
        #print('L: '+str(l))
        for s in S:
            #print('S: '+str(s))
            true_partworths = []
            for t in range(num_true_partworths):
                true_partworths.append(rng.multivariate_normal(l*init_mu,s*init_Sig))
            
            gumbel_errors = [[[np.random.gumbel(0,1) for k in range(2)] for j in range(batch_size)] for i in range(num_true_partworths)]
            
            for i in range(num_random_batches):
                #Create a list for the batch that will store the final determinant value for each simulation
                #corresponding to each baseline partworth.
                batch_simulate_d_values = []
                
                #Simulate d-efficiency over baseline partworths
                for j in range(len(true_partworths)):
                #Each time we start with a new partworth, we must use the initial prior parameters.
                    mu = l*init_mu
                    Sig = s*init_Sig
                    
                    #Each simulation goes through the questions in the random batch.
                    for k in range(batch_size):
                    #Set x and y
                        x = batch_set[i][k][0]
                        y = batch_set[i][k][1]
                
                        #These temp variables will be used in the choice model below in case the user prefers y over x.
                        x_temp = x
                        y_temp = y
                        
                        gum_x = gumbel_errors[j][k][0]
                        gum_y = gumbel_errors[j][k][1]
                        #See preference between two products
                        if (np.dot(true_partworths[j],np.array(y)) + gum_y) >= (np.dot(true_partworths[j],np.array(x)) + gum_x):
                            x = y_temp
                            y = x_temp
                            
                        #Perform moment matching after choice is made.
                        [mu, Sig] = moment_matching_update(x,y,mu,Sig)
                        
                    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
                    #of the final covariance matrix.
                    batch_simulate_d_values.append(np.sqrt(np.linalg.det(Sig)))
                    
                #We average the d-values from the simulation for a batch and store it in a list.
                average_d_error.append(np.mean(batch_simulate_d_values))
                
    return average_orthogonality, maximum_orthogonality, average_question_mean, average_question_variance, L_mu, S_Sig, init_sqrt_determinant, average_d_error

In [ ]:
#This function is used to create a set of batch designs and evaluate their D-error (in a sequential manner). 
#We save the D-error of each batch in a list.

def random_batch_D_error(init_mu,init_Sig,batch_size,num_random_batches,true_partworths, gumbel_error_terms):
    #init_mu: This is the initial expectation of the partworths.
    #init_Sig: This is the initial covariance matrix of the partworths.
    #batch_size: This is the number of questions in each batch.
    #num_random_batches: This is the number of random batches that we will generate.
    #true_partworths: This is the true/baseline partworths we will use to evaluate the d-error of a design.
    #gumbel_error_terms: This is a list of gumbel errors terms used in evaluating the d-error of a design. This 
    #list should have dimension (true_partworths x batch_size), where each entry is a list with two randomly generated
    #gumbel terms. We use the same true partworths and error terms for evaluating the d-error of each randomly generated 
    #design.
    
    attr_num = len(init_mu)
    
    average_d_error = []
    
    #Create a list of all products.
    prod_list = product_diff_list(attr_num)
    
    #Construct the set of batch designs
    batch_set = [[] for i in range(num_random_batches)]
    for i in range(num_random_batches):
        random_question_matrix = random.sample(prod_list,batch_size)
        for m in range(batch_size):
            #random_question_vec = random.sample(prod_list,1)[0]
            #[x,y] = question_extractor(random_question_vec)
            [x,y] = question_extractor(random_question_matrix[m])
            batch_set[i].append([x,y])
        
            
    #Evaluate the d-error
    #true_partworths = []
    #for t in range(num_true_partworths):
                #true_partworths.append(rng.multivariate_normal(init_mu,init_Sig))
    num_true_partworths = len(true_partworths)
    
    for i in range(num_random_batches):
                #Create a list for the batch that will store the final determinant value for each simulation
                #corresponding to each baseline partworth.
                batch_simulate_d_values = []
                #print('random_batch_number: ' + str(i))
                #Simulate d-efficiency over baseline partworths
                for j in range(num_true_partworths):
                #Each time we start with a new partworth, we must use the initial prior parameters.
                    mu = init_mu
                    Sig = init_Sig
                    
                    #Each simulation goes through the questions in the random batch.
                    for k in range(batch_size):
                    #Set x and y
                        x = batch_set[i][k][0]
                        y = batch_set[i][k][1]
                
                        #These temp variables will be used in the choice model below in case the user prefers y over x.
                        x_temp = x
                        y_temp = y
                        
                        #See preference between two products.
                        gum_x = gumbel_error_terms[j][k][0]#np.random.gumbel(0,1)
                        gum_y = gumbel_error_terms[j][k][1]#np.random.gumbel(0,1)
                        if (np.dot(true_partworths[j],np.array(y)) + gum_y) >= (np.dot(true_partworths[j],np.array(x)) + gum_x):
                            x = y_temp
                            y = x_temp
                            
                        #Perform moment matching after choice is made.
                        [mu, Sig] = moment_matching_update(x,y,mu,Sig)
                        
                    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
                    #of the final covariance matrix.
                    batch_simulate_d_values.append(np.sqrt(np.linalg.det(Sig)))
                    
                #We average the d-values from the simulation for a batch and store it in a list.
                average_d_error.append(np.mean(batch_simulate_d_values))
                
    return average_d_error

In [ ]:
#Fitting the models batch_AO and batch_MO.
rng = np.random.default_rng(100) 
np.random.seed(100)
random.seed(100)

#signal to noise ratio. 
#1 - LOW: multiply expectation by 0.25 and covariance by 4.0
#2 - REG: multiply expectation by 1.0 and covariance by 1.0
#3 - HIGH: multiply expectation by 4.0 and covariance by 0.25
snr = int(sys.argv[1])


#Prior type
#1 - homogeneous expectation and identity covariance matrix
#2 - heterogeneous expectation and KMS covariance matrix
prior_type = int(sys.argv[2])

if snr == 1:
    if prior_type == 1:
        init_mu_fit = 0.25*np.array(6*[1.0])
        init_Sig_fit = 4.0*np.identity(6)
    if prior_type == 2:
        init_mu_fit = 0.25*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        init_Sig_fit = 4.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 2:
    if prior_type == 1:
        init_mu_fit = 1.0*np.array(6*[1.0])
        init_Sig_fit = 1.0*np.identity(6)
    if prior_type == 2:
        init_mu_fit = 1.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        init_Sig_fit = 1.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 3:
    if prior_type == 1:
        init_mu_fit = 4.0*np.array(6*[1.0])
        init_Sig_fit = 0.25*np.identity(6)
    if prior_type == 2:
        init_mu_fit = 4.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        init_Sig_fit = 0.25*(1.25*KMS_Matrix(6,-0.5))
    
batch_size_fit = 4
    
L_fit = [0.5,1.0,2.0]
S_fit = [0.5,1.0,2.0]

num_random_batches_fit = 1000
num_true_partworths_fit = 50

In [ ]:
#Generate the data in order to estimate the parameters of the AO and MO models
average_orthogonality_fit, maximum_orthogonality_fit, average_question_mean_fit, average_question_variance_fit, L_mu_fit, S_Sig_fit, init_sqrt_determinant_fit, average_d_error_fit = norm_AO_MO_data_generation(init_mu_fit, init_Sig_fit, batch_size_fit, L_fit, S_fit, num_random_batches_fit, num_true_partworths_fit)

In [ ]:
#Create a dataframe of the generated data for fitting the parameters of the AO and MO models
df_fit = pd.DataFrame(list(zip(average_orthogonality_fit, maximum_orthogonality_fit, average_question_mean_fit, average_question_variance_fit, L_mu_fit, S_Sig_fit, init_sqrt_determinant_fit, average_d_error_fit)),
                  columns =['Avg_Orth', 'Max_Orth', 'Avg_Quest_Mean', 'Avg_Quest_Var', 'L_mu_norm', 'S_Sig_norm', 'Init_Sqrt_Det', 'D_err'])

In [ ]:
#Add some new columns to the dataset. We mean-center the independent variables to attempt to reduce VIF. This will not affect the value of
#of the coefficients, except for the intercept.
df_fit['log_norm_derr'] = np.log(np.divide(np.array(df_fit['D_err']),np.array(df_fit['Init_Sqrt_Det'])))
df_fit['cent_norm_AM'] = np.divide(np.array(df_fit['Avg_Quest_Mean']),np.array(df_fit['L_mu_norm'])) - np.mean(np.divide(np.array(df_fit['Avg_Quest_Mean']),np.array(df_fit['L_mu_norm'])))
df_fit['cent_norm_AV'] = np.divide(np.array(df_fit['Avg_Quest_Var']),np.array(df_fit['S_Sig_norm'])) - np.mean(np.divide(np.array(df_fit['Avg_Quest_Var']),np.array(df_fit['S_Sig_norm'])))
df_fit['cent_norm_AO'] = np.divide(np.array(df_fit['Avg_Orth']),np.array(df_fit['S_Sig_norm'])) - np.mean(np.divide(np.array(df_fit['Avg_Orth']),np.array(df_fit['S_Sig_norm'])))
df_fit['cent_norm_MO'] = np.divide(np.array(df_fit['Max_Orth']),np.array(df_fit['S_Sig_norm'])) - np.mean(np.divide(np.array(df_fit['Max_Orth']),np.array(df_fit['S_Sig_norm'])))

df_fit['cent_L_mu_norm'] = df_fit['L_mu_norm'] - np.mean(np.array(df_fit['L_mu_norm']))
df_fit['cent_S_Sig_norm'] = df_fit['S_Sig_norm'] - np.mean(np.array(df_fit['S_Sig_norm']))

In [ ]:
#Save resulting file as a CSV.
if snr == 1:
    if prior_type == 1:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_mu025_Sig4Ident_batchsize4_L_05_1_2_S_05_1_2_nrb_1000_ntp_50_v3.csv')
    if prior_type == 2:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_lowsnr_muhet_SigKMS_batchsize4_L_05_1_2_S_05_1_2_nrb_1000_ntp_50_v3.csv')
if snr == 2:
    if prior_type == 1:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_mu1_Sig1Ident_batchsize4_L_05_1_2_S_05_1_2_nrb_1000_ntp_50_v3.csv')
    if prior_type == 2:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_medsnr_muhet_SigKMS_batchsize4_L_05_1_2_S_05_1_2_nrb_1000_ntp_50_v3.csv')
if snr == 3:
    if prior_type == 1:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_mu4_Sig025Ident_batchsize4_L_05_1_2_S_05_1_2_nrb_1000_ntp_50_v3.csv')
    if prior_type == 2:
        df_fit.to_csv('MIPvEnum_Normalized_AO_MO_Model_Data_highsnr_muhet_SigKMS_batchsize4_L_05_1_2_S_05_1_2_nrb_1000_ntp_50_v3.csv')

In [ ]:
#Model with AO
model_AO = sm.formula.ols(formula = "log_norm_derr ~  cent_norm_AM + cent_norm_AV + cent_norm_AO + cent_L_mu_norm + cent_S_Sig_norm", data = df_fit).fit()
parameter_est_AO = model_AO.params

In [ ]:
#Model with MO
model_MO = sm.formula.ols(formula = "log_norm_derr ~  cent_norm_AM + cent_norm_AV + cent_norm_MO + cent_L_mu_norm + cent_S_Sig_norm", data = df_fit).fit()
parameter_est_MO = model_MO.params

In [ ]:
#Save model parameters in txt file
if snr == 1:
    params_est = np.array([parameter_est_AO[1],parameter_est_AO[2],parameter_est_AO[3],parameter_est_MO[1],parameter_est_MO[2],parameter_est_MO[3]])
    if prior_type == 1:
        np.savetxt('MIPvEnum_modelparams_lowsnr_v3.txt',params_est)
    if prior_type == 2:
        np.savetxt('MIPvEnum_modelparams_lowsnr_muhet_SigKMS_v3.txt', params_est)
if snr == 2:
    params_est = np.array([parameter_est_AO[1],parameter_est_AO[2],parameter_est_AO[3],parameter_est_MO[1],parameter_est_MO[2],parameter_est_MO[3]])
    if prior_type == 1:
        np.savetxt('MIPvEnum_modelparams_medsnr_v3.txt',params_est)
    if prior_type == 2:
        np.savetxt('MIPvEnum_modelparams_medsnr_muhet_SigKMS_v3.txt', params_est)
if snr == 3:
    params_est = np.array([parameter_est_AO[1],parameter_est_AO[2],parameter_est_AO[3],parameter_est_MO[1],parameter_est_MO[2],parameter_est_MO[3]])
    if prior_type == 1:
        np.savetxt('MIPvEnum_modelparams_highsnr_v3.txt',params_est)
    if prior_type == 2:
        np.savetxt('MIPvEnum_modelparams_highsnr_muhet_SigKMS_v3.txt', params_est)

In [ ]:
#Experiment settings:
#Settings for experiment
rng = np.random.default_rng(100)
np.random.seed(100)
random.seed(100)

    
if snr == 1:
    if prior_type == 1:
        mu_exp = 0.25*np.array(6*[1.0])
        Sig_exp = 4.0*np.identity(6)
    if prior_type == 2:
        mu_exp = 0.25*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        Sig_exp = 4.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 2:
    if prior_type == 1:
        mu_exp = 1.0*np.array(6*[1.0])
        Sig_exp = 1.0*np.identity(6)
    if prior_type == 2:
        mu_exp = 1.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        Sig_exp = 1.0*(1.25*KMS_Matrix(6,-0.5))
if snr == 3:
    if prior_type == 1:
        mu_exp = 4.0*np.array(6*[1.0])
        Sig_exp = 0.25*np.identity(6)
    if prior_type == 2:
        mu_exp = 4.0*np.array([-0.25,0.5,-0.75,1.0,-1.25,1.5])
        Sig_exp = 0.25*(1.25*KMS_Matrix(6,-0.5))

batch_size_exp = 4
num_random_batches_exp = 10000        

true_partworth_exp = []

num_true_partworths_exp = 100
#There will be 100 true partworths
for t in range(num_true_partworths_exp):
    true_partworth_exp.append(rng.multivariate_normal(mu_exp,Sig_exp))
    
#Create list of gumbel error terms
gumbel_error_terms_exp = [[[np.random.gumbel(0,1) for k in range(2)] for j in range(batch_size_exp)] for i in range(num_true_partworths_exp)]
print(gumbel_error_terms_exp[0])   

AO_alpha_exp = parameter_est_AO[1]
AO_kappa_exp = parameter_est_AO[2]
AO_gamma_exp = parameter_est_AO[3]

MO_alpha_exp = parameter_est_MO[1]
MO_kappa_exp = parameter_est_MO[2]
MO_gamma_exp = parameter_est_MO[3]



In [ ]:
#Create list of d-error of random batch designs
tic = time.perf_counter()
d_err_list = random_batch_D_error(mu_exp,Sig_exp,batch_size_exp,num_random_batches_exp,true_partworth_exp,gumbel_error_terms_exp)
toc = time.perf_counter()
time_meas = toc - tic

In [ ]:
#Create Batch_AO design and evaluate its D-error
[AO_batch,AO_Ortho] = batch_design_AO(mu_exp,Sig_exp,batch_size_exp,AO_alpha_exp,AO_kappa_exp,AO_gamma_exp,t_lim = 100)

In [ ]:
#Create Batch_MO design and evaluate its D-error
[MO_batch, MO_Ortho] = batch_design_MO(mu_exp,Sig_exp,batch_size_exp,MO_alpha_exp,MO_kappa_exp,MO_gamma_exp,t_lim = 100)

In [ ]:
#Evaluate the d-error of the Batch_AO design

#Create a list for the batch that will store the final determinant value for each simulation
#corresponding to each baseline partworth.\
np.random.seed(100) #Add this so that gumbel error terms 
batch_AO_d_values = []
                
#Simulate d-efficiency over baseline partworths
for j in range(len(true_partworth_exp)):
    #Each time we start with a new partworth, we must use the initial prior parameters.
    mu_AO = mu_exp
    Sig_AO = Sig_exp
                    
    #Each simulation goes through the questions in the random batch.
    for k in range(batch_size_exp):
        #Set x and y
        x_AO = AO_batch[k][0]
        y_AO = AO_batch[k][1]
                
        #These temp variables will be used in the choice model below in case the user prefers y over x.
        x_temp_AO = x_AO
        y_temp_AO = y_AO
        
        gum_x_AO = gumbel_error_terms_exp[j][k][0]
        gum_y_AO = gumbel_error_terms_exp[j][k][1]
        #See preference between two products.
        if (np.dot(true_partworth_exp[j],np.array(y_AO)) + gum_y_AO) >= (np.dot(true_partworth_exp[j],np.array(x_AO)) + gum_x_AO):
            x_AO = y_temp_AO
            y_AO = x_temp_AO
                            
        #Perform moment matching after choice is made.
        [mu_AO, Sig_AO] = moment_matching_update(x_AO,y_AO,mu_AO,Sig_AO)
                        
    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
    #of the final covariance matrix.
    batch_AO_d_values.append(np.sqrt(np.linalg.det(Sig_AO)))
                    
#We average the d-values from the simulation for the AO batch and save it as a variable.
batch_AO_d_error = np.mean(batch_AO_d_values)

In [ ]:
#Evaluate the d-error of the Batch_MO design

#Create a list for the batch that will store the final determinant value for each simulation
#corresponding to each baseline partworth.
np.random.seed(100) #Add this so that gumbel error terms will be the same as those used in AO.
batch_MO_d_values = []
                
#Simulate d-efficiency over baseline partworths
for j in range(len(true_partworth_exp)):
    #Each time we start with a new partworth, we must use the initial prior parameters.
    mu_MO = mu_exp
    Sig_MO = Sig_exp
                    
    #Each simulation goes through the questions in the random batch.
    for k in range(batch_size_exp):
        #Set x and y
        x_MO = MO_batch[k][0]
        y_MO = MO_batch[k][1]
                
        #These temp variables will be used in the choice model below in case the user prefers y over x.
        x_temp_MO = x_MO
        y_temp_MO = y_MO
        
        gum_x_MO = gumbel_error_terms_exp[j][k][0]
        gum_y_MO = gumbel_error_terms_exp[j][k][1]
        #See preference between two products. 
        if (np.dot(true_partworth_exp[j],np.array(y_MO)) + gum_y_MO) >= (np.dot(true_partworth_exp[j],np.array(x_MO)) + gum_x_MO):
            x_MO = y_temp_MO
            y_MO = x_temp_MO
                            
        #Perform moment matching after choice is made.
        [mu_MO, Sig_MO] = moment_matching_update(x_MO,y_MO,mu_MO,Sig_MO)
                        
    #After the questionnaire for a baseline partworth is complete, we append the square root of the determinant
    #of the final covariance matrix.
    batch_MO_d_values.append(np.sqrt(np.linalg.det(Sig_MO)))
                    
#We average the d-values from the simulation for the AO batch and save it as a variable.
batch_MO_d_error = np.mean(batch_MO_d_values)

In [ ]:
batch_AO_success = [1 if batch_AO_d_error < rand_batch_d_error else 0 for rand_batch_d_error in d_err_list]
batch_MO_success = [1 if batch_MO_d_error < rand_batch_d_error else 0 for rand_batch_d_error in d_err_list]

In [ ]:
#Save d-error information and success information in a csv
batch_information_list = np.array([d_err_list,batch_AO_success,batch_MO_success]).T
batch_information_list_df = pd.DataFrame(batch_information_list, columns = ['D-errors', 'Batch AO < D-error', 'Batch MO < D-error'])

if snr == 1:
    if prior_type == 1:
        batch_information_list_df.to_csv('MIPvsEnum_lowsnr_d_error_batch_comparison_list_v3.csv')
    if prior_type == 2:
        batch_information_list_df.to_csv('MIPvsEnum_lowsnr_d_error_batch_comparison_list_muhet_SigKMS_v3.csv')
if snr == 2:
    if prior_type == 1:
        batch_information_list_df.to_csv('MIPvsEnum_mediumsnr_d_error_batch_comparison_list_v3.csv')
    if prior_type == 2:
        batch_information_list_df.to_csv('MIPvsEnum_mediumsnr_d_error_batch_comparison_list_muhet_SigKMS_v3.csv')
if snr == 3:
    if prior_type == 1:
        batch_information_list_df.to_csv('MIPvsEnum_highsnr_d_error_batch_comparison_list_v3.csv')
    if prior_type == 2:
        batch_information_list_df.to_csv('MIPvsEnum_highsnr_d_error_batch_comparison_list_muhet_SigKMS_v3.csv')

In [ ]:
#Construct estimates and confidence interval
AO_prop_est = np.mean(batch_AO_success)
MO_prop_est = np.mean(batch_MO_success)

AO_CI_upper = AO_prop_est + 1.96*np.sqrt(AO_prop_est*(1-AO_prop_est)/len(batch_AO_success))
AO_CI_lower = AO_prop_est - 1.96*np.sqrt(AO_prop_est*(1-AO_prop_est)/len(batch_AO_success))

MO_CI_upper = MO_prop_est + 1.96*np.sqrt(MO_prop_est*(1-MO_prop_est)/len(batch_MO_success))
MO_CI_lower = MO_prop_est - 1.96*np.sqrt(MO_prop_est*(1-MO_prop_est)/len(batch_MO_success))

In [ ]:
#Save confidence interval information, time for iterating over random batches, and d-error of batches
if snr == 1:
    CI_information = np.array(['batch_AO_d_error: ' + str(batch_AO_d_error),'AO_prop_est: ' +str(AO_prop_est),'AO_CI_lower: ' + str(AO_CI_lower),'AO_CI_upper: ' + str(AO_CI_upper),'batch_MO_d_error: ' + str(batch_MO_d_error),'MO_prop_est: ' + str(MO_prop_est),'MO_CI_lower:' + str(MO_CI_lower),'MO_CI_upper: ' + str(MO_CI_upper),'time for enumeration: ' + str(time_meas)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_CIinfo_lowsnr_v3.txt',CI_information,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_CIinfo_lowsnr_muhet_SigKMS_v3.txt',CI_information,fmt='%s')
if snr == 2:
    CI_information = np.array(['batch_AO_d_error: ' + str(batch_AO_d_error),'AO_prop_est: ' +str(AO_prop_est),'AO_CI_lower: ' + str(AO_CI_lower),'AO_CI_upper: ' + str(AO_CI_upper),'batch_MO_d_error: ' + str(batch_MO_d_error),'MO_prop_est: ' + str(MO_prop_est),'MO_CI_lower:' + str(MO_CI_lower),'MO_CI_upper: ' + str(MO_CI_upper),'time for enumeration: ' + str(time_meas)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_CIinfo_mediumsnr_v3.txt',CI_information,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_CIinfo_mediumsnr_muhet_SigKMS_v3.txt',CI_information,fmt='%s')
if snr == 3:
    CI_information = np.array(['batch_AO_d_error: ' + str(batch_AO_d_error),'AO_prop_est: ' +str(AO_prop_est),'AO_CI_lower: ' + str(AO_CI_lower),'AO_CI_upper: ' + str(AO_CI_upper),'batch_MO_d_error: ' + str(batch_MO_d_error),'MO_prop_est: ' + str(MO_prop_est),'MO_CI_lower:' + str(MO_CI_lower),'MO_CI_upper: ' + str(MO_CI_upper),'time for enumeration: ' + str(time_meas)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_CIinfo_highsnr_v3.txt',CI_information,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_CIinfo_highsnr_muhet_SigKMS_v3.txt',CI_information,fmt='%s')

In [ ]:
#Save information regarding the batch design.
if snr == 1:
    batch_info = np.array(['AO_batch: '+str(AO_batch),'AO_Orth: '+str(AO_Ortho),'MO_batch: '+str(MO_batch),'MO_Orth: '+str(MO_Ortho)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_batchinfo_lowsnr_v3.txt',batch_info,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_batchinfo_lowsnr_muhet_SigKMS_v3.txt',batch_info,fmt='%s')
if snr == 2:
    batch_info = np.array(['AO_batch: '+str(AO_batch),'AO_Orth: '+str(AO_Ortho),'MO_batch: '+str(MO_batch),'MO_Orth: '+str(MO_Ortho)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_batchinfo_mediumsnr_v3.txt',batch_info,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_batchinfo_mediumsnr_muhet_SigKMS_v3.txt',batch_info,fmt='%s')
if snr == 3:
    batch_info = np.array(['AO_batch: '+str(AO_batch),'AO_Orth: '+str(AO_Ortho),'MO_batch: '+str(MO_batch),'MO_Orth: '+str(MO_Ortho)])
    if prior_type == 1:
        np.savetxt('MIPvEnum_batchinfo_highsnr_v3.txt',batch_info,fmt='%s')
    if prior_type == 2:
        np.savetxt('MIPvEnum_batchinfo_highsnr_muhet_SigKMS_v3.txt',batch_info,fmt='%s')